# BiteMe | Train

This notebook includes the most important part of the project - the modelling. The notebook tests methodologies for training, and in it the chosen algorithm is decided. Validation also occurs before final testing, which is conducted in the test notebook. This stage is highly iterative, so all model artefacts, logs and configurations are recorded and saved to disk automatically. This initial setup of what will eventually become MLOps for the final product will be really useful, and helps keep track of what is successful and what isn't.

Models to try:

 - [DenseNet121](https://github.com/Cadene/pretrained-models.pytorch#torchvision)
 - [DenseNet161](https://github.com/Cadene/pretrained-models.pytorch#torchvision)
 - [DenseNet169](https://github.com/Cadene/pretrained-models.pytorch#torchvision)
 - [DenseNet201](https://github.com/Cadene/pretrained-models.pytorch#torchvision)
 - [DenseNet201](https://github.com/Cadene/pretrained-models.pytorch#torchvision)
 - [DualPathNet68](https://github.com/Cadene/pretrained-models.pytorch#dualpathnetworks)
 - [DualPathNet92](https://github.com/Cadene/pretrained-models.pytorch#dualpathnetworks)
 - [DualPathNet98](https://github.com/Cadene/pretrained-models.pytorch#dualpathnetworks)
 - [DualPathNet107](https://github.com/Cadene/pretrained-models.pytorch#dualpathnetworks)
 - [DualPathNet113](https://github.com/Cadene/pretrained-models.pytorch#dualpathnetworks)
 - [FBResNet152](https://github.com/Cadene/pretrained-models.pytorch#facebook-resnet)
 - [InceptionResNetV2](https://github.com/Cadene/pretrained-models.pytorch#inception)
 - [InceptionV4](https://github.com/Cadene/pretrained-models.pytorch#inception)
 - [NASNet-A-Large](https://github.com/Cadene/pretrained-models.pytorch#nasnet)
 - [PNASNet-5-Large](https://github.com/Cadene/pretrained-models.pytorch#pnasnet)
 - [PolyNet](https://github.com/Cadene/pretrained-models.pytorch#polynet)
 - [ResNeXt101_32x4d](https://github.com/Cadene/pretrained-models.pytorch#resnext)
 - [ResNeXt101_64x4d](https://github.com/Cadene/pretrained-models.pytorch#resnext)
 - [ResNet101](https://github.com/Cadene/pretrained-models.pytorch#torchvision)
 - [ResNet152](https://github.com/Cadene/pretrained-models.pytorch#torchvision)
 - [ResNet50](https://github.com/Cadene/pretrained-models.pytorch#torchvision)
 - [SENet154](https://github.com/Cadene/pretrained-models.pytorch#senet)
 - [SE-ResNet50](https://github.com/Cadene/pretrained-models.pytorch#senet)
 - [SE-ResNet101](https://github.com/Cadene/pretrained-models.pytorch#senet)
 - [SE-ResNet152](https://github.com/Cadene/pretrained-models.pytorch#senet)
 - [SE-ResNeXt50_32x4d](https://github.com/Cadene/pretrained-models.pytorch#senet)
 - [SE-ResNeXt101_32x4d](https://github.com/Cadene/pretrained-models.pytorch#senet)
 - [SqueezeNet1_0](https://github.com/Cadene/pretrained-models.pytorch#torchvision)
 - [SqueezeNet1_1](https://github.com/Cadene/pretrained-models.pytorch#torchvision)
 - [VGG16](https://github.com/Cadene/pretrained-models.pytorch#torchvision)
 - [VGG19](https://github.com/Cadene/pretrained-models.pytorch#torchvision)
 - [VGG16_BN](https://github.com/Cadene/pretrained-models.pytorch#torchvision)
 - [VGG19_BN](https://github.com/Cadene/pretrained-models.pytorch#torchvision)
 - [Xception](https://github.com/Cadene/pretrained-models.pytorch#xception)


- efficientnet_b0
- efficientnet_b1
- efficientnet_b2
- efficientnet_b3
- efficientnet_b4
- efficientnet_b5

Initial model work is done by using simple, typical image recognition models (CNN architectures) to see how effective these models can be for the problem. Although I don't expect them to be particularly successful, it's important to establish baselines and take a holistic approach to modelling when it's possible.

In [1]:
# Basic imports
import pandas as pd
import numpy as np
import os
import sys
from argparse import ArgumentParser
import datetime
from time import time
import gc

# Data visualisation
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn

# Image processing
import cv2
import albumentations as A
import imgaug as ia
import imgaug.augmenters as iaa

# Model evaluation
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_auc_score, f1_score

import torch
import pretrainedmodels
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint

# Local imports
sys.path.append("..")
from utils.dataset import generate_transforms, generate_dataloaders
from models.models import se_resnet50
from utils.loss_function import CrossEntropyLossOneHot
from utils.lrs_scheduler import WarmRestart, warm_restart
from utils.utils import read_images, augs, get_augs, seed_reproducer, init_logger
from utils.constants import *

plt.rcParams["figure.figsize"] = (14, 8)

In [2]:
# Define directories
base_dir_path = "../"

data_dir_path = os.path.join(base_dir_path, "data")
data_preprocessed_dir_path = os.path.join(data_dir_path, "preprocessed")
data_preprocessed_train_dir_path = os.path.join(data_dir_path, "preprocessed/train")

data_dir = os.listdir(data_dir_path)
data_preprocessed_dir = os.listdir(data_preprocessed_dir_path)
data_preprocessed_train_dir = os.listdir(data_preprocessed_train_dir_path)

metadata_preprocessed_path = os.path.join(data_preprocessed_dir_path, "metadata.csv")
metadata = pd.read_csv(metadata_preprocessed_path)
# Subset to train only
metadata = metadata.loc[metadata.split == "train"]

metadata.head()

,img_name,img_path,label,split
0,7059b14d2aa03ed6c4de11afa32591995181d31c.jpg,../data/cleaned/none/7059b14d2aa03ed6c4de11afa...,none,train
1,ea1b100b581fcdb7ddfae52cc62347a99e304ba4.jpg,../data/cleaned/none/ea1b100b581fcdb7ddfae52cc...,none,train
3,6eac051b9c45ff6821ec8675216f371711b7cea9.jpg,../data/cleaned/none/6eac051b9c45ff6821ec86752...,none,train
4,fc72767f8520df9b2b83941077dc0ee013eb9399.jpg,../data/cleaned/none/fc72767f8520df9b2b8394107...,none,train
5,cf812984268e2aec9a167d3ebe1026f610dd862b.jpg,../data/cleaned/none/cf812984268e2aec9a167d3eb...,none,train


In [3]:
# Read in train images
X_train = read_images(
    data_dir_path=data_preprocessed_train_dir_path, 
    rows=ROWS, 
    cols=COLS, 
    channels=CHANNELS, 
    write_images=False, 
    output_data_dir_path=None,
    verbose=VERBOSE
)

# Get labels
y_train = np.array(pd.get_dummies(metadata["label"]))

Reading images from: ../data/preprocessed/train
Rows set to 1024
Columns set to 1024
Channels set to 3
Writing images is set to: False
Reading images...


100%|███████████████████████████████████████████| 23/23 [00:03<00:00,  7.54it/s]


Image reading complete.
Image array shape: (192, 1024, 1024, 3)


## Set Parameters

In [4]:
# Choose augmentations to use in preprocessing
# For full list see helpers.py
#augs_to_select = [
#    "Resize",
#    "HorizontalFlip", 
#    "VerticalFlip",
#    "Normalize"
#]
## Subset augs based on those selected
#AUGS = dict((aug_name, augs[aug_name]) for aug_name in augs_to_select)


def init_hparams():
    """
    Initialise hyperparameters for modelling.
    
    Returns
    ---------
    hparams : argparse.Namespace
        Parsed hyperparameters
    """
    parser = ArgumentParser(add_help=False)
    parser.add_argument("-backbone", "--backbone", type=str, default=MODEL_NAME)
    parser.add_argument("-device_name", type=str, default=DEVICE_NAME)
    parser.add_argument("--gpus", default=[0])
    parser.add_argument("--n_workers", type=int, default=N_WORKERS)
    parser.add_argument("--image_size", nargs="+", default=[ROWS, COLS])
    parser.add_argument("--seed", type=int, default=SEED)
    parser.add_argument("--min_epochs", type=int, default=MIN_EPOCHS)
    parser.add_argument("--max_epochs", type=int, default=MAX_EPOCHS)
    parser.add_argument("--patience", type=str, default=PATIENCE)    
    parser.add_argument("-tbs", "--train_batch_size", type=int, default=TRAIN_BATCH_SIZE)
    parser.add_argument("-vbs", "--val_batch_size", type=int, default=VAL_BATCH_SIZE)
    parser.add_argument("--n_splits", type=int, default=N_SPLITS)
    parser.add_argument("--test_size", type=float, default=TEST_SIZE)
    parser.add_argument("--precision", type=int, default=PRECISION)
    parser.add_argument("--gradient_clip_val", type=float, default=GRADIENT_CLIP_VAL)
    parser.add_argument("--verbose", type=str, default=VERBOSE)
    parser.add_argument("--log_dir", type=str, default=LOG_DIR)
    parser.add_argument("--log_name", type=str, default=LOG_NAME)
    
    
    try:
        hparams, unknown = parser.parse_known_args()
    except:
        hparams, unknown = parser.parse_args([])

    if len(hparams.gpus) == 1:
        hparams.gpus = [int(hparams.gpus[0])]
    else:
        hparams.gpus = [int(gpu) for gpu in hparams.gpus]

    hparams.image_size = [int(size) for size in hparams.image_size]
    
    return hparams

### Create Model

In [5]:
class CoolSystem(pl.LightningModule):
    def __init__(self, hparams):
        super().__init__()
        self.hparams = hparams

        seed_reproducer(self.hparams.seed)

        self.model = se_resnet50()
        self.criterion = CrossEntropyLossOneHot()
        self.logger_kun = init_logger(
            hparams.log_name, 
            hparams.log_dir
        )

    def forward(self, x):
        return self.model(x)

    def configure_optimizers(self):
        self.optimizer = torch.optim.Adam(
            self.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0
        )
        self.scheduler = WarmRestart(self.optimizer, T_max=10, T_mult=1, eta_min=1e-5)
        return [self.optimizer], [self.scheduler]

    def training_step(self, batch, batch_idx):
        step_start_time = time()
        images, labels, data_load_time = batch

        scores = self(images)
        loss = self.criterion(scores, labels)

        data_load_time = torch.sum(data_load_time)

        return {
            "loss": loss,
            "data_load_time": data_load_time,
            "batch_run_time": torch.Tensor([time() - step_start_time + data_load_time]).to(
                data_load_time.device
            ),
        }

    def training_epoch_end(self, outputs):
        # outputs is the return of training_step
        train_loss_mean = torch.stack([output["loss"] for output in outputs]).mean()
        self.data_load_times = torch.stack([output["data_load_time"] for output in outputs]).sum()
        self.batch_run_times = torch.stack([output["batch_run_time"] for output in outputs]).sum()

        self.current_epoch += 1
        if self.current_epoch < (self.trainer.max_epochs - 4):
            self.scheduler = warm_restart(self.scheduler, T_mult=2)

        return {"train_loss": train_loss_mean}

    def validation_step(self, batch, batch_idx):
        step_start_time = time()
        images, labels, data_load_time = batch
        data_load_time = torch.sum(data_load_time)
        scores = self(images)
        loss = self.criterion(scores, labels)

        # must return key -> val_loss
        return {
            "val_loss": loss,
            "scores": scores,
            "labels": labels,
            "data_load_time": data_load_time,
            "batch_run_time": torch.Tensor([time() - step_start_time + data_load_time]).to(
                data_load_time.device
            ),
        }

    def validation_epoch_end(self, outputs):
        # compute loss
        val_loss_mean = torch.stack([output["val_loss"] for output in outputs]).mean()
        self.data_load_times = torch.stack([output["data_load_time"] for output in outputs]).sum()
        self.batch_run_times = torch.stack([output["batch_run_time"] for output in outputs]).sum()

        # compute roc_auc
        scores_all = torch.cat([output["scores"] for output in outputs]).cpu()
        labels_all = torch.round(torch.cat([output["labels"] for output in outputs]).cpu())
        val_roc_auc = torch.tensor(roc_auc_score(labels_all, scores_all))

        # terminal logs
        self.logger_kun.info(
            f"{self.hparams.fold_i}-{self.current_epoch} | "
            f"lr : {self.scheduler.get_lr()[0]:.6f} | "
            f"val_loss : {val_loss_mean:.4f} | "
            f"val_roc_auc : {val_roc_auc:.4f} | "
            f"data_load_times : {self.data_load_times:.2f} | "
            f"batch_run_times : {self.batch_run_times:.2f}"
        )

        return {"val_loss": val_loss_mean, "val_roc_auc": val_roc_auc}

## Cross Validation

In [6]:
# Initialise hyperparameters
hparams = init_hparams()
# Initialise logger
logger = init_logger(hparams.log_name, hparams.log_dir)

# Create transform pipeline
transforms = generate_transforms(hparams.image_size)

# List for validation scores 
valid_roc_auc_scores = []

# Initialise cross validation
folds = StratifiedKFold(n_splits=hparams.n_splits, shuffle=True, random_state=hparams.seed)

# Start cross validation
for fold_i, (train_index, val_index) in enumerate(folds.split(metadata[["img_path"]], metadata[["label"]])):
    hparams.fold_i = fold_i
    # Split train images and validation sets
    train_data = metadata.iloc[train_index][["img_path", "label"]].reset_index(drop=True)
    train_data = pd.get_dummies(train_data, columns=["label"], prefix="", prefix_sep="")

    val_data = metadata.iloc[val_index][["img_path", "label"]].reset_index(drop=True)
    val_data = pd.get_dummies(val_data, columns=["label"], prefix="", prefix_sep="")
    
    logger.info(f"Fold {fold_i} num train records: {train_data.shape[0]}")
    logger.info(f"Fold {fold_i} num val records: {val_data.shape[0]}")
    
    train_dataloader, val_dataloader = generate_dataloaders(hparams, train_data, val_data, transforms)
    
    checkpoint_callback = ModelCheckpoint(
        monitor="val_roc_auc",
        save_top_k=2,
        mode="max",
        filepath=os.path.join(
            hparams.log_dir, 
            hparams.log_name, 
            f"fold={fold_i}" + "-{epoch}-{val_loss:.4f}-{val_roc_auc:.4f}"
        )
    )
    
    early_stop_callback = EarlyStopping(
        monitor="val_roc_auc", 
        patience=hparams.patience, 
        mode="max", 
        verbose=hparams.verbose
    )
    
    # Instance Model, Trainer and train model
    model = CoolSystem(hparams)
    trainer = pl.Trainer(
        gpus=hparams.gpus,
        min_epochs=hparams.min_epochs,
        max_epochs=hparams.max_epochs,
        early_stop_callback=early_stop_callback,
        checkpoint_callback=checkpoint_callback,
        progress_bar_refresh_rate=0,
        precision=hparams.precision,
        num_sanity_val_steps=0,
        profiler=False,
        weights_summary=None,
        gradient_clip_val=hparams.gradient_clip_val,
        default_root_dir=os.path.join(hparams.log_dir, hparams.log_name)
    )
    
    # Fit model
    trainer.fit(model, train_dataloader, val_dataloader)
        
    # Generate model output for validation 
    #preds = model(
    #    torch.from_numpy(X_train[val_index]).permute(0, 3, 1, 2).float()
    #)
#
    ## Create activation output
    #log_softmax = torch.nn.LogSoftmax(dim=-1)
#
    ## Convert raw output to probabilities
    #preds = np.exp(log_softmax(preds).detach().numpy())
#
    ## Create df with img paths and predicted label probs
    #scores_df = pd.DataFrame(preds, columns=val_data.columns[1:])
    #scores_df = pd.merge(
    #    metadata.iloc[val_index, 1:3].reset_index(drop=True),
    #    scores_df, 
    #    left_index=True,
    #    right_index=True
    #)
    ## Write predictions to log
    #scores_df.to_csv(
    #    os.path.join(hparams.log_dir, hparams.log_name, f"{hparams.log_name}_preds_fold_{fold_i}.csv"),
    #    index=False
    #)
    
    # Save val scores
    valid_roc_auc_scores.append(checkpoint_callback.best)
    
    # Cleanup
    del model
    gc.collect()
    torch.cuda.empty_cache()
    
valid_roc_auc_scores = [i.item() for i in valid_roc_auc_scores]

# Add val scores to csv with all scores
if os.path.isfile("../logs/scores.csv") == False:
    pd.DataFrame(columns=["name", "scores", "mean_score"]).to_csv("../logs/scores.csv", index=False)
    
# Append to current scores csv
all_scores_df = pd.concat([
    pd.read_csv("../logs/scores.csv"),
    pd.DataFrame.from_dict(
        {
            "name": [hparams.log_name],
            "scores": [valid_roc_auc_scores],
            "mean_score": [np.mean(valid_roc_auc_scores)]
        }
    )],
    ignore_index=True
)
# Write all scores df to csv
all_scores_df.to_csv("../logs/scores.csv", index=False)

logger.info(f"Best scores: {valid_roc_auc_scores}")
logger.info("Training complete.")

[2022-09-12 18:59:26] 3786432242.py[  25] : INFO  Fold 0 num train records: 128
[2022-09-12 18:59:26] 3786432242.py[  26] : INFO  Fold 0 num val records: 64
Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /home/edwardsims/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.
[2022-09-12 19:00:04] 2105417623.py[  84] : INFO  0-0 | lr : 0.001000 | val_loss : 2.0778 | val_roc_auc : 0.5328 | data_load_times : 34.32 | batch_run_times : 34.48
[2022-09-12 19:00:35] 2105417623.py[  84] : INFO  0-1 | lr : 0.000976 | val_loss : 2.0893 | val_roc_auc : 0.5040 | data_load_times : 35.36 | batch_run_times : 35.47
[2022-09-12 19:01:05] 2105417623.py[  84] : INFO  0-2 | lr : 0.000905 | val_loss : 2.0651 | val_roc_auc : 0.5660 | data_load_times : 37.83 | batch_run_times : 37.93
[2022-09-12 19:01:35] 2105417623.py[  84] : INFO  0-3 | lr : 0.000796 | val_loss : 2.0520 | val_roc_auc : 0.5628 | data_load_times : 39.47 | batch_run_times : 39.64
[2022-09-12 19:02:06] 2105417623.py[  84] : INFO  0-4 | lr : 0.000658 | val_loss : 2.0721 | val_roc_auc : 0.5243 | data_load_times : 38.40 | batch_run_times : 38.51
[2022-09-12 19:02:37] 2105417623.py[  84] : IN

[2022-09-12 19:23:25] 2105417623.py[  84] : INFO  1-4 | lr : 0.000658 | val_loss : 2.0767 | val_roc_auc : 0.4904 | data_load_times : 38.54 | batch_run_times : 38.67
[2022-09-12 19:23:57] 2105417623.py[  84] : INFO  1-5 | lr : 0.000505 | val_loss : 2.0494 | val_roc_auc : 0.5942 | data_load_times : 41.41 | batch_run_times : 41.53
[2022-09-12 19:24:29] 2105417623.py[  84] : INFO  1-6 | lr : 0.000352 | val_loss : 2.0679 | val_roc_auc : 0.5385 | data_load_times : 36.22 | batch_run_times : 36.34
[2022-09-12 19:25:01] 2105417623.py[  84] : INFO  1-7 | lr : 0.000214 | val_loss : 2.0396 | val_roc_auc : 0.5632 | data_load_times : 39.86 | batch_run_times : 39.98
[2022-09-12 19:25:33] 2105417623.py[  84] : INFO  1-8 | lr : 0.000105 | val_loss : 2.1009 | val_roc_auc : 0.5120 | data_load_times : 41.21 | batch_run_times : 41.32
[2022-09-12 19:26:04] 2105417623.py[  84] : INFO  1-9 | lr : 0.000034 | val_loss : 2.1045 | val_roc_auc : 0.4878 | data_load_times : 40.12 | batch_run_times : 40.23
[2022-09-1

[2022-09-12 19:45:27] 2105417623.py[  84] : INFO  2-6 | lr : 0.000352 | val_loss : 2.0675 | val_roc_auc : 0.5051 | data_load_times : 37.79 | batch_run_times : 37.91
[2022-09-12 19:45:58] 2105417623.py[  84] : INFO  2-7 | lr : 0.000214 | val_loss : 2.0339 | val_roc_auc : 0.5937 | data_load_times : 39.65 | batch_run_times : 39.79
[2022-09-12 19:46:31] 2105417623.py[  84] : INFO  2-8 | lr : 0.000105 | val_loss : 2.0724 | val_roc_auc : 0.5631 | data_load_times : 39.46 | batch_run_times : 39.56
[2022-09-12 19:47:02] 2105417623.py[  84] : INFO  2-9 | lr : 0.000034 | val_loss : 2.0722 | val_roc_auc : 0.5399 | data_load_times : 34.92 | batch_run_times : 35.04
[2022-09-12 19:47:32] 2105417623.py[  84] : INFO  2-10 | lr : 0.001000 | val_loss : 2.1230 | val_roc_auc : 0.5293 | data_load_times : 36.49 | batch_run_times : 36.62
[2022-09-12 19:48:03] 2105417623.py[  84] : INFO  2-11 | lr : 0.000976 | val_loss : 2.0135 | val_roc_auc : 0.5865 | data_load_times : 36.94 | batch_run_times : 37.06
[2022-09